## ML을 하기 위해 해야 하는 것들

- 데이터 나누기 : Train, Validation, Test
- 데이터 전처리 하기
  - 탐색적 자료 분석 (EDA) - 데이터 한번 들여다보기
  - feature engineering

## train data와 test 데이터는 하나의 모판으로 다뤄야 한다

## 이유

- Train과 Test 데이터 셋의 각 컬럼에는 같은 전처리를 적용해야 함 (같은 형태의 데이터가 들어가야 함)
  - 한 판을 먼저 만들어서 train과 test 데이터를 같이 다뤄주는 것이 좋다.
- 초기 데이터 전처리시 규칙을 만들어야 함
- 데이터의 분포를 좀 더 넓게 볼 수 있음
- 실제 서비스에서는 모델 생성시에 사용한 전처리를 그대로 활용함

In [1]:
import pandas as pd
import os

## Titanic

- kaggle - tatanic 검색 - notebooks - EDA검색하면 좋은 코드 볼 수 있음

#### Load Dataset

In [23]:
DATA_DIR = './titanic'
data_files = reversed([os.path.join(DATA_DIR, filename) for filename in os.listdir(DATA_DIR)])
df_list = []
for filename in data_files:
    df_list.append(pd.read_csv(filename))
df = pd.concat(df_list, sort=False)
df = df.reset_index(drop=True)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## 모판에서 y데이터만 제거

- 따로 떼서 관리하든 안 하든 자유이지만, 이번 경우에는 따로 뗴어서 관리

In [24]:
number_of_train_dataset = df['Survived'].notnull().sum()  # train data의 개수
number_of_test_dataset = df['Survived'].isnull().sum()  # test data의 개수
y_true = df.pop('Survived')[:number_of_train_dataset]  # train data까지 슬라이싱

---

## 데이터 처리의 전략

- 모판은 흔들지 않는다
- 하나의 셀은 다시 실행해도 그 결과가 보장되야 한다. (특히 jupyter notebook 쓸 때 주의사항)
- 전처리가 완료된 후 함수화한다. (merge 함수 필수)
- column 이름은 list로 관리한다. (직접 입력 X)
- 데이터는 타입별로 분리해서 관리하기
- 데이터 노트 작성하기!
  - 컬럼의 수가 아주 많아지면, 그 컬럼이 어떤 것을 의미하는지, 이 컬럼을 써도 될지 말아도 될지 또한 모르기 때문에 note를 하나 만들어줘야 한다. 

## 데이터 노트

- 데이터에 대한 처리 내용 및 방향을 정리한 노트 (내가 어떻게 분석을 진행할 것인지)
- 기본적인 전처리 방향과 방법들을 정리함
- 데이터에 대한 아이디어를 정리와 업데이트

## 기본적인 데이터 현황 파악 코드

- df.dtypes
- df.info()
- df.isnull().sum()
- df.describe()
- df.head(2).T

## Data Cleansing issues

- 데이터가 빠진 경우 (결측치의 처리)
- 라벨링된 데이터 (category) 데이터의 처리
  - ex) string type의 데이터 처리는 어떻게?
- 데이터의 scale의 차이가 매우 크게 나는 경우
  - -> scaling

In [26]:
df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [27]:
df.head(2).T

,0,1
PassengerId,1,2
Pclass,3,1
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th..."
Sex,male,female
Age,22.0,38.0
SibSp,1,1
Parch,0,0
Ticket,A/5 21171,PC 17599
Fare,7.25,71.2833
Cabin,NaN,C85


In [28]:
df.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64